In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from typing import List

In [2]:
from selenium_management import start_driver, open_link, get_wait_element, get_wait_elements, close_driver, get_links

In [3]:
driver = start_driver()

### АВТОРИЗАЦИЯ

In [4]:
# Авторизация
def auth(driver: webdriver, username: str, password: str):
    username = 'odintsovmaxim4@gmail.com'
    password = 'password1923DF!'
    open_link(driver=driver, link="https://www.instagram.com/")
    username_element = get_wait_element(driver=driver, by=By.NAME, searched_elem='username')
    username_element.send_keys(username)

    password_element = get_wait_element(driver=driver, by=By.NAME, searched_elem='password')
    password_element.send_keys(password)

    enter_element = get_wait_element(driver=driver, by=By.XPATH, searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div/section/main/article/div[2]/div[1]/div[2]/div/form/div/div[3]/button/div')
    enter_element.click()
    time.sleep(5)
    return True

auth(driver=driver, username='odintsovmaxim4@gmail.com', password='password1923DF!')

True

### Парсинг ссылок на посты

In [59]:
# Переход на страницу с постами
def turn_to_posts_page(driver: webdriver, query: str):
    link = f'https://www.instagram.com/explore/search/keyword/?q={query}'
    open_link(driver=driver, link=link)

def get_post_links(driver: webdriver, wait_time: int = 5, max_scrolls: int = 10) -> set:
    """Скроллинг страниц и поиск ссылок"""

    last_height = driver.execute_script("return document.body.scrollHeight")
    post_links = set()
    max_scrolls += 1
    for i in range(1, max_scrolls):
        # Скролл страницы
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)
        
        # Получение ссылок
        if i % 2 == 0 or i == max_scrolls-1:
            posts_parent_elelemt = get_wait_element(driver=driver, by=By.XPATH, searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[2]')
            new_links = get_links(parent_element=posts_parent_elelemt)
            print(f'Найдены новые ссылки на посты: {new_links}')
            post_links.update(new_links)
            
        # Проверка конца страницы
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    return post_links

In [60]:
turn_to_posts_page(driver=driver, query='%23rapmusic') # '%23r' == '#'
post_links = get_post_links(driver, max_scrolls=1)

Найдены новые ссылки на посты: ['https://www.instagram.com/p/CVSghF9oX6D/', 'https://www.instagram.com/p/C_GzW5fy43J/', 'https://www.instagram.com/p/C69P6qASiuI/', 'https://www.instagram.com/p/DCqLsrtzJIg/', 'https://www.instagram.com/p/C69QKazvW5f/', 'https://www.instagram.com/p/C69PW9zSeqW/', 'https://www.instagram.com/p/C4TsnmdR6-w/', 'https://www.instagram.com/p/C4FuMoJJuPa/', 'https://www.instagram.com/p/C5JEAxEppQ2/', 'https://www.instagram.com/p/C6DBwmco5Zl/', 'https://www.instagram.com/p/DDBO7dgRjZd/', 'https://www.instagram.com/p/C2lEbG7BZKT/', 'https://www.instagram.com/p/C3C6AakyV_z/', 'https://www.instagram.com/p/DDPq0vqTIdm/', 'https://www.instagram.com/p/C17lSNeP2P-/', 'https://www.instagram.com/p/DAoKFL7IpWe/', 'https://www.instagram.com/p/C2S3PHSvCUN/', 'https://www.instagram.com/p/C75cqkji8jO/']


In [61]:
post_links = list(post_links)
print(f"Найдено {len(post_links)} ссылок на посты")
for href in post_links:
    print(href)

Найдено 18 ссылок на посты
https://www.instagram.com/p/CVSghF9oX6D/
https://www.instagram.com/p/C5JEAxEppQ2/
https://www.instagram.com/p/C6DBwmco5Zl/
https://www.instagram.com/p/C_GzW5fy43J/
https://www.instagram.com/p/DDBO7dgRjZd/
https://www.instagram.com/p/C69PW9zSeqW/
https://www.instagram.com/p/C4TsnmdR6-w/
https://www.instagram.com/p/DAoKFL7IpWe/
https://www.instagram.com/p/C2lEbG7BZKT/
https://www.instagram.com/p/C69P6qASiuI/
https://www.instagram.com/p/DDPq0vqTIdm/
https://www.instagram.com/p/C17lSNeP2P-/
https://www.instagram.com/p/C69QKazvW5f/
https://www.instagram.com/p/C75cqkji8jO/
https://www.instagram.com/p/C4FuMoJJuPa/
https://www.instagram.com/p/C2S3PHSvCUN/
https://www.instagram.com/p/DCqLsrtzJIg/
https://www.instagram.com/p/C3C6AakyV_z/


### Парсинг ссылок на аккаунты

In [62]:
def accounts_parsing(post_links: List[str]) -> set:
    account_links = set()
    for post_link in post_links:
        find_links = False
        open_link(driver=driver, link=post_link)
        
        # Поиск одного аккаунта в посте
        accounts_parent = get_wait_element(
            driver=driver, 
            by=By.XPATH, 
            searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[1]/div/div/div/span/span',
            delay=5,
            attempts=1,
            is_error=False
        )
        if accounts_parent:
            print('Найдено в "Поиск одного аккаунта в посте"')
            find_links = True
            
        # Поиск "ещё" в указанных аккаунтах поста
        if find_links is False:
            modal_accounts_element = get_wait_element(
                driver=driver, 
                by=By.XPATH, 
                searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div/div/span/span[2]/a/span',
                delay=5,
                attempts=1,
                is_error=False
            )
            if modal_accounts_element and 'ещё' in modal_accounts_element.text:
                modal_accounts_element.click()  # Открытие модального окна с ссылками на аккаунты
                accounts_parent = get_wait_element(
                    driver=driver, 
                    by=By.XPATH, 
                    searched_elem='/html/body/div[6]/div[1]/div/div[2]/div/div/div',
                    sleep=3,
                    delay=5,
                    attempts=1,
                    is_error=False
                )
                if accounts_parent:
                    print('Найдено в "Поиск "ещё" в указанных аккаунтах поста"')
                    find_links = True
        
        # Поиск двух указанных аккаунтов в посте
        if find_links is False:
            accounts_parent = get_wait_element(
                driver=driver, 
                by=By.XPATH, 
                searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div/div',
                delay=5,
                attempts=1,
                is_error=False
            )
            if accounts_parent:
                print('Найдено в "Поиск двух указанных аккаунтов в посте"')
                find_links = True
        
        # Поиск ссылок на аккаунты из родительского элемента
        if find_links is True:
            new_links = get_links(accounts_parent)
            print(f'Найдены ссылки: {new_links}')
            account_links.update(new_links)
            
    return account_links


In [63]:
# links = accounts_parsing(['https://www.instagram.com/p/C4TsnmdR6-w/'])
# links
account_links = accounts_parsing(post_links)

Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/rappirates/']
Найдено в "Поиск "ещё" в указанных аккаунтах поста"
Найдены ссылки: ['https://www.instagram.com/the.rapdaily/', 'https://www.instagram.com/the.rapdaily/', 'https://www.instagram.com/the.rapdaily/', 'https://www.instagram.com/slimeraptv/', 'https://www.instagram.com/slimeraptv/', 'https://www.instagram.com/slimeraptv/', 'https://www.instagram.com/raptrending.us/', 'https://www.instagram.com/raptrending.us/', 'https://www.instagram.com/raptrending.us/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/fortylvofficial/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://www.instagram.com/officialnoconscience/']
Найдено в "Поиск двух указанных аккаунтов в посте"
Найдены ссылки: ['https://www.instagram.com/soulknowledge_2x/', 'https://www.instagram.com/soulknowledge/']
Найдено в "Поиск одного аккаунта в посте"
Найдены ссылки: ['https://ww

In [64]:
account_links = list(account_links)
print(f"Найдено {len(account_links)} ссылок на аккаунты")
for href in account_links:
    print(href)

Найдено 25 ссылок на аккаунты
https://www.instagram.com/dosemusicplugtv/
https://www.instagram.com/lamonte_______/
https://www.instagram.com/music___records__/
https://www.instagram.com/rapinsoul/
https://www.instagram.com/4ktrey_squad/
https://www.instagram.com/soulknowledge_2x/
https://www.instagram.com/jltheengineer/
https://www.instagram.com/detroitrap.news/
https://www.instagram.com/soulknowledge/
https://www.instagram.com/fortylvofficial/
https://www.instagram.com/raptrending.us/
https://www.instagram.com/dc_therapper/
https://www.instagram.com/musicsfavoritecolour/
https://www.instagram.com/dc_therapperx/
https://www.instagram.com/officialnoconscience/
https://www.instagram.com/usarap_tv/
https://www.instagram.com/slimeraptv/
https://www.instagram.com/jj.4ktrey/
https://www.instagram.com/the.rapdaily/
https://www.instagram.com/mrtwosides11/
https://www.instagram.com/dttheartist/
https://www.instagram.com/4ktrey_swag/
https://www.instagram.com/bunny_therapper/
https://www.instagr

### Парсинг страничек

In [65]:
def extract_original_link(raw_link):
    """
    Функция извлекает оригинальную ссылку
    """
    from urllib.parse import urlparse, parse_qs
    
    parsed_url = urlparse(raw_link)
    query_params = parse_qs(parsed_url.query)
    orig_link = query_params.get("u", [""])[0]
    if orig_link:
        
        return query_params.get("u", [""])[0]
    return raw_link

In [66]:
import re

# Функция для преобразования текста в числовой формат
def parse_number_accurate(text):
    number_map = {
        'тыс.': 1000,
        'млн': 1000000,
        'млн.': 1000000
    }
    number = re.findall(r'[\d\s,]+', text)
    multiplier = 1
    for key, value in number_map.items():
        if key in text:
            multiplier = value
            break
    if number:
        clean_number = number[0].replace(' ', '').replace(',', '.')
        return int(float(clean_number) * multiplier)
    return 0

# Основная функция для распределения данных
def parse_activity_data(data_list):
    result = {'posts': 0, 'subscribers': 0, 'subscriptions': 0}
    for item in data_list:
        if 'публикац' in item:
            result['posts'] = parse_number_accurate(item)
        elif 'подписчик' in item:
            result['subscribers'] = parse_number_accurate(item)
        elif any(word in item for word in ['подписок', 'подписки', 'подписка']):
            result['subscriptions'] = parse_number_accurate(item)
    return result


def parsing_account_info(driver: webdriver, account_link: str) -> dict:
    """
    Парсинг основной информации об аккаунте
    """
    driver.get(account_link)
    time.sleep(3)
    
    links = None
    all_description = None
    activity_attrs = {}
    
    #################################
    ### Парсинг описания аккаунта ###
    #################################
    account_description_parent_element = get_wait_element(  # Элемент со всей указанной информацией об аккаунте
        driver=driver,
        by=By.XPATH,
        searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[4]',
        delay=3,
        attempts=1,
        is_error=False
    )
    if account_description_parent_element:
        # Парсинг описания аккаунта
        all_description = account_description_parent_element.text

        # Парсинг ссылок, указанных в контактах аккаунта
        account_links_button_element = get_wait_element(  # Кнопка с указанными ссылками
            driver=account_description_parent_element,
            by=By.TAG_NAME,
            searched_elem='button',
            delay=3,
            attempts=1,
            is_error=False
        )
        if account_links_button_element and '+' in account_links_button_element.text:  # Если указано несколько ссылок 
            account_links_button_element.click()  # Открытие модального окна с указанными ссылками
            link_parent_element = get_wait_element(  # Модальное окно с указанными ссылками
                driver=account_links_button_element, 
                by=By.XPATH, 
                searched_elem='/html/body/div[6]/div[1]/div/div[2]/div/div/div/div',
                delay=4,
                attempts=1,
                sleep=2,
                is_error=False
            )
            
            find_links = get_links(link_parent_element)
            links = [extract_original_link(find_link) for find_link in find_links]
        if not links:  # Если в указанных ссылках нет "+ n"
            find_links = get_links(account_description_parent_element)
            links = [extract_original_link(find_link) for find_link in find_links]
    print(f'\n___________________\nСсылка на аккаунт: {account_link}\nall_description: {all_description}\nСсылки: {links}')

    ###################################
    ### Парсинг активности аккаунта ###
    ###################################
    activity_account_parent_element = get_wait_element(  # Родительский элемент с кол-вом постов, подписчиков и подписок
        driver=driver, 
        by=By.XPATH, 
        searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/header/section[3]',
        delay=3,
        attempts=1,
        is_error=False
    )
    if activity_account_parent_element:
        activity_account_attributes_elements = get_wait_elements(  # Родительский элемент с кол-вом постов, подписчиков и подписок
            driver=activity_account_parent_element, 
            by=By.TAG_NAME, 
            searched_elem='li',
            delay=3,
            attempts=1,
            is_error=False
        )
        if activity_account_attributes_elements:
            activity_list = [activity_account_attributes_element.text for activity_account_attributes_element in activity_account_attributes_elements]
            activity_attrs = parse_activity_data(activity_list)
            print(f'Найдены атрибуты активности: {activity_attrs}')

    account_info = {'account_link': account_link, 'all_description': all_description, 'links': links} | activity_attrs
    return account_info


In [67]:
accounts = []
for account_link in account_links:
# for account_link in ['https://www.instagram.com/akashi_cruz/']:
    accounts_info = parsing_account_info(driver=driver, account_link=account_link)
    accounts.append(accounts_info)
    
accounts


___________________
Ссылка на аккаунт: https://www.instagram.com/dosemusicplugtv/
all_description: 𝓓𝓮𝓮𝓹 𝓒𝓱𝓪𝓷𝓭
dosemusicplugtv
СМИ/информационное агентство
Dm For Promo/Shootout!
Best For Upcoming Artist 🥷
YouTube!🔔/SoundCloud/Spotify/Instagram/ All Service Available... 
ещё
Ссылки: ['https://www.threads.net/@dosemusicplugtv?xmt=AQGz8cg4ZQGd-8nQFPTpGwSKPvQ7G3zcWonyTL_0FsmKfII']
Найдены атрибуты активности: {'posts': 1228, 'subscribers': 50500, 'subscriptions': 1100}

___________________
Ссылка на аккаунт: https://www.instagram.com/lamonte_______/
all_description: Музыкант/группа
🎨Art | music 🎼
Here to blend my two passions
Need Music Production or 2D Animation DM or 📩
linktr.ee/Lamnt247
Ссылки: ['https://linktr.ee/Lamnt247?fbclid=PAZXh0bgNhZW0CMTEAAaZ52PCWCMKZDN_D2VqEmfUktIEqCOZ6ikNo4_YFjoew-G1XyRc6pHE20AQ_aem_Z8kgZajmPZr5K--6hfAufA']
Найдены атрибуты активности: {'posts': 32, 'subscribers': 14200, 'subscriptions': 891}

___________________
Ссылка на аккаунт: https://www.instagram.com/

[{'account_link': 'https://www.instagram.com/dosemusicplugtv/',
  'all_description': '𝓓𝓮𝓮𝓹 𝓒𝓱𝓪𝓷𝓭\ndosemusicplugtv\nСМИ/информационное агентство\nDm For Promo/Shootout!\nBest For Upcoming Artist 🥷\nYouTube!🔔/SoundCloud/Spotify/Instagram/ All Service Available... \nещё',
  'links': ['https://www.threads.net/@dosemusicplugtv?xmt=AQGz8cg4ZQGd-8nQFPTpGwSKPvQ7G3zcWonyTL_0FsmKfII'],
  'posts': 1228,
  'subscribers': 50500,
  'subscriptions': 1100},
 {'account_link': 'https://www.instagram.com/lamonte_______/',
  'all_description': 'Музыкант/группа\n🎨Art | music 🎼\nHere to blend my two passions\nNeed Music Production or 2D Animation DM or 📩\nlinktr.ee/Lamnt247',
  'links': ['https://linktr.ee/Lamnt247?fbclid=PAZXh0bgNhZW0CMTEAAaZ52PCWCMKZDN_D2VqEmfUktIEqCOZ6ikNo4_YFjoew-G1XyRc6pHE20AQ_aem_Z8kgZajmPZr5K--6hfAufA'],
  'posts': 32,
  'subscribers': 14200,
  'subscriptions': 891},
 {'account_link': 'https://www.instagram.com/music___records__/',
  'all_description': '💰music___records💰\nМузыкант/гр

### Просмотр и запись данных

In [68]:
import pandas as pd
df = pd.DataFrame.from_dict(accounts)
pd.set_option('display.width', None)  # Автоматическая ширина вывода
pd.set_option('display.max_columns', None)  # Показывать все столбцы
df

,account_link,all_description,links,posts,subscribers,subscriptions
0,https://www.instagram.com/dosemusicplugtv/,𝓓𝓮𝓮𝓹 𝓒𝓱𝓪𝓷𝓭\ndosemusicplugtv\nСМИ/информационно...,[https://www.threads.net/@dosemusicplugtv?xmt=...,1228,50500,1100
1,https://www.instagram.com/lamonte_______/,Музыкант/группа\n🎨Art | music 🎼\nHere to blend...,[https://linktr.ee/Lamnt247?fbclid=PAZXh0bgNhZ...,32,14200,891
2,https://www.instagram.com/music___records__/,💰music___records💰\nМузыкант/группа\nTHE #1COMM...,[https://m.youtube.com/playlist?list=OLAK5uy_n...,161,43400,170
3,https://www.instagram.com/rapinsoul/,❁ Rap In Soul ❁ 🔵\nrapinsoul\nСМИ/информационн...,[https://www.threads.net/@rapinsoul?xmt=AQGznp...,316,127000,722
4,https://www.instagram.com/4ktrey_squad/,MEME GOD 💚💯\n4ktrey_squad\n#1 fan page🐍\nFaste...,[https://www.threads.net/@4ktrey_squad?xmt=AQG...,3251,22700,173
5,https://www.instagram.com/soulknowledge_2x/,For The Culture\nA Backup For The Best HipHop/...,[https://instagram.com/soulknowledge_2x?igshid...,1016,16100,147
6,https://www.instagram.com/jltheengineer/,JL Brown\njltheengineer\nЗвукоинженер\nMulti-P...,[https://www.threads.net/@jltheengineer?xmt=AQ...,2173,25400,7484
7,https://www.instagram.com/detroitrap.news/,detroitrapnews\ndetroitrap.news\nMusician/band...,[https://www.threads.net/@detroitrap.news?xmt=...,768,130000,1206
8,https://www.instagram.com/soulknowledge/,For The Culture\nHipHop/R&B & Everything In Th...,[],2650,98600,205
9,https://www.instagram.com/fortylvofficial/,Private Mirambeaux\nhe/him/it\nfortylvofficial...,[https://www.threads.net/@fortylvofficial?xmt=...,5,23900,6570


In [69]:
import json
with open('new_data.json', 'w') as f:
    data = json.dump(accounts, indent=4, ensure_ascii=False, fp=f)

In [ ]:
with open('data.json', 'r') as f:
    loaded_data = json.load(f)
loaded_data

### Запись данных в таблицу

In [70]:
from openpyxl import Workbook

In [71]:
wb = Workbook()
ws = wb.active
ws.title = "Instagram Data"

# Записываем заголовки столбцов
headers = ["Account Link", "Description", "Links"]
ws.append(headers)

# Заполняем данные
for account_item in accounts:
    account_link = account_item['account_link']
    all_description = account_item['all_description']
    links = "\n".join(account_item['links'])  # Преобразуем список ссылок в строку
    ws.append([account_link, all_description, links])

# Сохраняем файл
wb.save("new)instagram_data.xlsx")
print("Данные успешно записаны в файл 'instagram_data.xlsx'")

Данные успешно записаны в файл 'instagram_data.xlsx'


In [72]:
from excel import PyXLWriter

In [74]:
###############################
### ЗАПИСЬ ДАННЫХ В ТАБЛИЦУ ###
###############################

pyxl = PyXLWriter(colors=2)
# Хедеры
pyxl[1, 1] = "Ссылка на аккаунт"
pyxl[1, 2] = "Описание страницы"
pyxl[1, 3] = "Ссылки из контактов"
pyxl[1, 4] = "Кол-во постов"
pyxl[1, 5] = "Кол-во подписчиков"
pyxl[1, 6] = "Кол-во подписок"
# Запись данных в таблицу
for rid, account_item in enumerate(accounts):
    r = rid + 2
    pyxl[r, 1] = account_item.get('account_link')
    pyxl[r, 2] = account_item.get('all_description')
    pyxl[r, 3] = "\n------------------------------\n".join(account_item.get('links'))
    pyxl[r, 4] = str(account_item.get('posts'))
    pyxl[r, 5] = str(account_item.get('subscribers'))
    pyxl[r, 6] = str(account_item.get('subscriptions'))
# Сохраняем файл
pyxl.save("new_instagram_data.xlsx")

In [ ]:
pyxl.ws.max_column